In [0]:
df=spark.read.csv("/Volumes/workspace/default/demo/server_logs.csv",header=True)
df.display()

<span style="color:red">
<b>Input Data set is attached in the assignment get the file from there.<br>
   
</span>

<b>Convert timestamp from string to DateTime.

In [0]:
# Write your code using the Spark Function
from pyspark.sql.functions import col, to_timestamp
df1 = df.withColumn("timestamp", to_timestamp(col("timestamp")))
display(df1)


<b> Extract the date from timestamp and add it as a new column date?

In [0]:
# Write your code using the Spark Function
from pyspark.sql.functions import to_date, col
df2 = df1.withColumn("Date", to_date(col("timestamp")))
df2.display()

<b> Find the number of logs each day.

In [0]:
# Write your code using the Spark Function
from pyspark.sql.functions import col
df3=df2.groupBy(col('Date')).count()
df3.display()
# Write your code using the Spark Function



<b> Extract hour from timestamp and analyze log count per hour.

In [0]:
from pyspark.sql.functions import hour, col

hourly = (
    df.withColumn("hour", hour(col("timestamp")))
      .groupBy("hour")
      .count()
      .orderBy("hour")
)

display(hourly)


<b> Identify the busiest hour of the day based on log count.

In [0]:
from pyspark.sql.functions import hour, col

busiest_hour = (
    df.withColumn("hour", hour(col("timestamp")))
      .groupBy("hour")
      .count()
      .orderBy(col("count").desc())
      .limit(1)
      
)

display(busiest_hour)


<b> Convert timestamp to a different string format: 'YYYY/MM/DD HH:mm'.

In [0]:
# Write your code using the Spark Function
from pyspark.sql.functions import col,to_date,lit
df7=df.withColumn('StringDate',lit('01-01-2024'))
dateformat="YYYY-MM-dd"
df8=df7.withColumn('Date',to_date(col('StringDate'),dateformat))
df8.show()

<b> Calculate the average response time for each URL.

In [0]:
# Write your code using the Spark Function
from pyspark.sql.functions import avg,col
df8 = df.select(avg(col('response_time_ms')))
df8.show()

<b> Filter logs with response time greater than a certain threshold.


In [0]:
# Write your code using the Spark Function
from pyspark.sql.functions import col

threshold = 100
df_slow = df.filter(col("response_time_ms").cast("double") > threshold)

display(df_slow)


<b> Find the top 5 most visited URLs.

In [0]:
# Write your code using the Spark Function# Write your code using the Spark Function
from pyspark.sql.functions import col
df9=df.withColumn('Top5url', col('url')).groupBy(col('url')).count().orderBy(col('count').desc()).limit(5)
df9.display()


<b> Calculate the day of the week for each log and count logs per day of the week.

In [0]:
from pyspark.sql.functions import date_format, col

df_dayofweek = df.withColumn("day_of_week", date_format(col("timestamp"), "EEEE"))
logs_per_dayofweek = df_dayofweek.groupBy("day_of_week").count().orderBy("day_of_week")

display(logs_per_dayofweek)

In [0]:
# Write your code using the Spark Function

<b> Identify logs with empty or null timestamp and count them.

In [0]:
from pyspark.sql.functions import col

df_null_empty = df.filter((col('timestamp').isNull()) | (col('timestamp') == ""))
count_null_empty = df_null_empty.count()
display(df_null_empty)
print(count_null_empty)

In [0]:
# Write your code using the Spark Function
from pyspark.sql.functions import col,when
df10=df.filter(col('timestamp').isNull()).count()
print(df10)

<b> Split user_agent into browser name and version and add as new columns.

In [0]:
# Write your code using the Spark Function
from pyspark.sql.functions import col,split
df11=df.withColumn('Newcolumn',split(col('user_agent'),'/'))
df11.show()

<b> Find the first and last log entry for each day

In [0]:
from pyspark.sql.functions import col, to_date, min as _min, max as _max

df_day = df.withColumn("day", to_date(col("timestamp")))

first_last = (
    df_day.groupBy("day")
          .agg(
              _min(col("timestamp")).alias("first_ts"),
              _max(col("timestamp")).alias("last_ts")
          ).limit(1)
          .orderBy("day")
)

display(first_last)


In [0]:
# Write your code using the Spark Function

<b> Extract the month from timestamp and analyze log count per month.

In [0]:
from pyspark.sql.functions import col, date_format

df_month = df.withColumn("month", date_format(col("timestamp"), "MMMM"))
logs_per_month = df_month.groupBy("month").count().orderBy("month")

display(logs_per_month)

In [0]:
# Write your code using the Spark Function

<b> Determine the average response time per hour of the day.

In [0]:
# Write your code using the Spark Function
from pyspark.sql.functions import to_date,date_format,avg
df13=df.withColumn('avg',date_format(col('timestamp'),"hh"))
df14=df13.select(avg(col('avg')))
df14.show()

<b> Create a new column showing the time of day (Morning, Afternoon, Evening, Night) based on timestamp

In [0]:
from pyspark.sql.functions import hour, col, when

df_time_of_day = df.withColumn(
    "time_of_day",
    when(hour(col("timestamp")).between(5, 11), "Morning")
    .when(hour(col("timestamp")).between(12, 16), "Afternoon")
    .when(hour(col("timestamp")).between(17, 20), "Evening")
    .otherwise("Night")
)

display(df_time_of_day)

In [0]:
# Write your code using the Spark Function

<b> Count the number of logs for each user_agent type.

In [0]:
# Write your code using the Spark Function
from pyspark.sql.functions import col
df15=df.withColumn('Numberoflogs',col('user_agent')).groupBy('user_agent').count()
df15.display()

<b> Find logs where the url contains a specific keyword (e.g., 'contact').

In [0]:
# Write your code using the Spark Function
from pyspark.sql.functions import col
df15=df.filter(col("url")=='/contact')
df15.display()

<b> Convert the timestamp to a Unix timestamp format.

In [0]:
from pyspark.sql.functions import unix_timestamp, col

df_unix = df.withColumn("unix_ts", unix_timestamp(col("timestamp")))
display(df_unix)

In [0]:
# Write your code using the Spark Function

<b> Calculate the time difference in minutes between consecutive logs.


In [0]:
from pyspark.sql.functions import to_timestamp, col, lag
from pyspark.sql.window import Window

df_ts = df.withColumn("timestamp_ts", to_timestamp(col("timestamp")))  # add format string if needed

w = Window.orderBy(col("timestamp_ts"))

df2 = (
    df_ts.withColumn("prev_ts", lag(col("timestamp_ts")).over(w))
         .withColumn(
             "diff_minutes",
             (col("timestamp_ts").cast("long") - col("prev_ts").cast("long")) / 60
         )
)

display(df2)


In [0]:
# Write your code using the Spark Function